#Sentiment Analysis

Sentiment Analysis:

**the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.**

The example we’ll use here is classifying movie reviews as either postive, negative or neutral.

*This guide is based on the following tensorflow tutorial: https://www.tensorflow.org/tutorials/text/text_classification_rnn*


## IMDB Movie review dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.
 




### Import and setup

In [1]:
# imports

%tensorflow_version 2.x
import tensorflow as tf
from keras.datasets import imdb
from keras.preprocessing import sequence
import os
import numpy as np


#setup

VOCAB_SIZE = 88584 # we have 88584 unique words(least common words) in this dataset
MAX_LEN = 250
BATCH_SIZE = 64


### Load dataset

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)


17473536/17464789 [==============================] - 0s 0us/step


In [3]:
# check shape of all train and test dataset
print(train_data.shape)
print(train_labels.shape) # (25000,) -- 1D with 25000 labels
print(test_data.shape)
print(test_labels.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [4]:
# look at review third

train_data[2] # we have different encoding for all the words, like: 106, 14 .......

[1,
 14,
 47,
 8,
 30,
 31,
 7,
 4,
 249,
 108,
 7,
 4,
 5974,
 54,
 61,
 369,
 13,
 71,
 149,
 14,
 22,
 112,
 4,
 2401,
 311,
 12,
 16,
 3711,
 33,
 75,
 43,
 1829,
 296,
 4,
 86,
 320,
 35,
 534,
 19,
 263,
 4821,
 1301,
 4,
 1873,
 33,
 89,
 78,
 12,
 66,
 16,
 4,
 360,
 7,
 4,
 58,
 316,
 334,
 11,
 4,
 1716,
 43,
 645,
 662,
 8,
 257,
 85,
 1200,
 42,
 1228,
 2578,
 83,
 68,
 3912,
 15,
 36,
 165,
 1539,
 278,
 36,
 69,
 44076,
 780,
 8,
 106,
 14,
 6905,
 1338,
 18,
 6,
 22,
 12,
 215,
 28,
 610,
 40,
 6,
 87,
 326,
 23,
 2300,
 21,
 23,
 22,
 12,
 272,
 40,
 57,
 31,
 11,
 4,
 22,
 47,
 6,
 2307,
 51,
 9,
 170,
 23,
 595,
 116,
 595,
 1352,
 13,
 191,
 79,
 638,
 89,
 51428,
 14,
 9,
 8,
 106,
 607,
 624,
 35,
 534,
 6,
 227,
 7,
 129,
 113]

In [5]:
# also length of all reviews are not same
print(len(train_data[0]))
print(len(train_data[2]))
print(len(train_data[0]))

218
141
218


###More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's **using padding** to make it equal to 250.





In [6]:
train_data = sequence.pad_sequences(train_data, MAX_LEN)
test_data = sequence.pad_sequences(test_data, MAX_LEN)

In [7]:
# now check revie 3rd agai which lenght was 141 so Keras must have added 0's padding on left side of interger

train_data[2]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     1,    14,    47,     8,    30,    31,     7,     4,
         249,   108,     7,     4,  5974,    54,    61,   369,    13,
          71,   149,

## Create Model

We'll use a **word embedding layer** as the first layer in our model and add a **LSTM layer** afterwards that feeds into **one dense node** to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [8]:
model = tf.keras.Sequential([
              tf.keras.layers.Embedding(VOCAB_SIZE, 32), # 32 dimension, passing interger to word embeddings layer to get more meaningful representation
              tf.keras.layers.LSTM(32), # LSTM will creat 32 dimension for each words
              tf.keras.layers.Dense(1, activation='sigmoid') # predict sentiment number < 0.5--> negative review, num> 0.5--> positive review
            ])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [10]:
# compile the model

model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['acc'])

# 'binary_crossentropy' --> can tell how far away we are from right probability 

# Model Training

In [11]:
model_train = model.fit(train_data, train_labels, epochs = 10, validation_split= 0.2)


# validation_split= 0.2 -- use 20% data from train for model evaluation

Epoch 1/10
625/625 [==============================] - 42s 59ms/step - loss: 0.4256 - acc: 0.8044 - val_loss: 0.3105 - val_acc: 0.8702
Epoch 2/10
625/625 [==============================] - 36s 57ms/step - loss: 0.2359 - acc: 0.9107 - val_loss: 0.2937 - val_acc: 0.8784
Epoch 3/10
625/625 [==============================] - 35s 57ms/step - loss: 0.1819 - acc: 0.9347 - val_loss: 0.3407 - val_acc: 0.8778
Epoch 4/10
625/625 [==============================] - 36s 57ms/step - loss: 0.1503 - acc: 0.9471 - val_loss: 0.3045 - val_acc: 0.8818
Epoch 5/10
625/625 [==============================] - 35s 57ms/step - loss: 0.1276 - acc: 0.9549 - val_loss: 0.3250 - val_acc: 0.8844
Epoch 6/10
625/625 [==============================] - 35s 57ms/step - loss: 0.1109 - acc: 0.9631 - val_loss: 0.3028 - val_acc: 0.8960
Epoch 7/10
625/625 [==============================] - 35s 56ms/step - loss: 0.0974 - acc: 0.9664 - val_loss: 0.3453 - val_acc: 0.8848
Epoch 8/10
625/625 [==============================] - 35s 57ms

# Model Evaluation
- with test data

In [15]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 13s 17ms/step - loss: 0.4263 - acc: 0.8584
[0.42625370621681213, 0.8583999872207642]


# Making predictions

let’s use our network to make predictions on our own reviews. 

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.


In [19]:
# Encoding:  text/word into integer 


word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text) # convert text into tokens(word sequences)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAX_LEN)[0] # first index to padded

text = "that movie was just amazing, so amazing" # 7 words, we get 7 integer encoding at the end of list : [0 0 0 ....... 12  17  13  40 477  35 477]
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [20]:
# Decoding: integer to word

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0 # no word there
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))


that movie was just amazing so amazing


In [21]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.8896567]
[0.40000808]
